In [50]:
import pandas as pd
import numpy as np

In [107]:
hil = pd.read_csv('../Data/Cost_of_living/HPI_AT_BDL_county.csv',
                 usecols=['FIPS code', 'State', 'County', 'HPI', 'Annual Change (%)'])
hil = hil.groupby(by='FIPS code', sort=False, ).last()
hil['Annual Change (%)'] = pd.qcut(pd.to_numeric(hil.iloc[:,-2],errors='coerce'), 10, labels=[i for i in range(1,11)], )
hil['HPI'] = pd.qcut(pd.to_numeric(hil.iloc[:,-1],errors='coerce'), 10, labels=[i for i in range(1,11)], )
hil.head(3)

,State,County,Annual Change (%),HPI
FIPS code,,,,
36061,NY,New York,NaN,NaN


In [99]:
tax = pd.read_csv('../Data/Cost_of_living/taxes.csv', 
                  skiprows=list(range(1,100,2)),
                 usecols=['State', 'Total Tax Burden'],)
tax['Total Tax Burden'] = pd.qcut(tax.iloc[:,-1].str.rstrip('%').astype('float'), 10, labels=[i for i in range(1,11)], )
tax= tax.dropna()
tax.tail(3)

,State,Total Tax Burden
47,Tennessee,1
48,Delaware,1
49,Alaska,1


In [108]:
xl = pd.ExcelFile('C:/Users/User/Documents/GitHub/AnotherTown/Data/Health/2020 County Health Rankings Data.xlsx')
income = pd.read_excel(xl, sheet_name= xl.sheet_names[-2], skiprows=[0], 
                    usecols=['FIPS','State','County','Median Household Income'])
income= income.dropna(axis=0, subset=['County'])
income['Median Household Income'] = pd.qcut(-1*income.iloc[:,-1], 10, labels=[i for i in range(1,11)], )
income.head(3)

,FIPS,State,County,Median Household Income
1,1001,Alabama,Autauga,3
2,1003,Alabama,Baldwin,3
3,1005,Alabama,Barbour,10


In [109]:
df = income.merge(hil.drop(axis=1, 
                           columns=['State','County']), 
               right_on='FIPS code', 
               left_on='FIPS')
df.head(3)

,FIPS,State,County,Median Household Income,Annual Change (%),HPI
0,1001,Alabama,Autauga,3,5,3
1,1003,Alabama,Baldwin,3,9,8
2,1005,Alabama,Barbour,10,1,4


In [110]:
df = df.merge(tax, on='State')

In [114]:
df['col_scores'] = [(row[-1]*2 + row[-2]*8 + row[-3]*2 )/12 for i, row in df.iterrows()]
df.head(3)
df.to_csv('../Data/Cleaned/COL_Raw.csv')

In [115]:
df.drop(columns=['Median Household Income','Annual Change (%)','HPI', 'Total Tax Burden']).to_csv('../Data/Cleaned/COL_Scores.csv')

In [113]:
df.head(3)
df[df['County']=='New York']

,FIPS,State,County,Median Household Income,Annual Change (%),HPI,Total Tax Burden,col_scores
1649,36061,New York,New York,1,NaN,NaN,10,NaN
